In [1]:
'''
จากบทความ "ลองเล่น Sentiment Analysis"
โดย Nuipin Decimo
https://bit.ly/3cMgR5a
'''

'\nจากบทความ "ลองเล่น Sentiment Analysis"\nโดย Nuipin Decimo\nhttps://bit.ly/3cMgR5a\n'

In [15]:
# ใช้ตัดคำภาษาไทย
import deepcut
# ใช้งาน regex
import re
# จัดการเกี่ยวกับ array
import numpy as np
# สำหรับทำ classify และทดสอบโมเดล
from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util
# สำหรับสร้างชุดข้อมูลสำหรับ train และ test เพื่อทดสอบประสิทธิภาพ
from sklearn.model_selection import KFold
import collections, itertools
# for save model
import pickle

In [7]:
# Thai Sentiment Text Analysis
# คลังข้อมูลสำหรับ Sentiment ภาษาไทย โดย นาย วรรณพงษ์ ภัททิยไพบูลย์
# https://github.com/PyThaiNLP/lexicon-thai

data_pos = [(line.strip(), 'pos') for line in open("pos.txt", 'r', encoding="utf8")]
data_neg = [(line.strip(), 'neg') for line in open("neg.txt", 'r', encoding="utf8")]
data_neutral = [(line.strip(), 'neutral') for line in open("neutral.txt", 'r', encoding="utf8")]

In [9]:
def split_words (sentence):
    return deepcut.tokenize(''.join(sentence.lower().split()))
sentences = [(split_words(sentence), sentiment) for (sentence, sentiment) in data_pos + data_neg]

In [10]:
def get_words_in_sentences(sentences):
    all_words = []
    for (words, sentiment) in sentences:
        all_words.extend(words)
    return all_words

In [11]:
def get_word_features(wordlist):
    wordlist = FreqDist(wordlist)
    word_features = [word[0] for word in wordlist.most_common()]
    return word_features

In [13]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [16]:
features_data = np.array(sentences)
# แบ่งข้อมูลเป็น 10 ชุด โดยไม่เรียง
k_fold = KFold(n_splits=10, random_state=1992, shuffle=True)
word_features = None
accuracy_scores = []
for train_set, test_set in k_fold.split(features_data):
    word_features = get_word_features(get_words_in_sentences(features_data[train_set].tolist()))
    train_features = apply_features(extract_features, features_data[train_set].tolist())
    test_features = apply_features(extract_features, features_data[test_set].tolist())
    classifier = NaiveBayesClassifier.train(train_features)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
    for i, (feats, label) in enumerate(test_features):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
    accuracy_score = util.accuracy(classifier, test_features)
    print('train: {} test: {}'.format(len(train_set), len(test_set)))
    print('=================== Results ===================')
    print('Accuracy {:f}'.format(accuracy_score))
    print('            Positive     Negative')
    print('F1         [{:f}     {:f}]'.format(
        f_measure(refsets['pos'], testsets['pos']),
        f_measure(refsets['neg'], testsets['neg'])
    ))
    print('Precision  [{:f}     {:f}]'.format(
        precision(refsets['pos'], testsets['pos']),
        precision(refsets['neg'], testsets['neg'])
    ))
    print('Recall     [{:f}     {:f}]'.format(
        recall(refsets['pos'], testsets['pos']),
        recall(refsets['neg'], testsets['neg'])
    ))
    print('===============================================\n')
    f = open('my_classifier.pickle', 'wb')
    pickle.dump(classifier, f)
    f.close()

C:\Users\porntip.k\AppData\Local\Temp/ipykernel_12660/2909416455.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features_data = np.array(sentences)


train: 956 test: 107
=================== Results ===================
Accuracy 0.728972
            Positive     Negative
F1         [0.602740     0.794326]
Precision  [0.916667     0.674699]
Recall     [0.448980     0.965517]

train: 956 test: 107
=================== Results ===================
Accuracy 0.785047
            Positive     Negative
F1         [0.741573     0.816000]
Precision  [0.804878     0.772727]
Recall     [0.687500     0.864407]

train: 956 test: 107
=================== Results ===================
Accuracy 0.747664
            Positive     Negative
F1         [0.666667     0.796992]
Precision  [0.964286     0.670886]
Recall     [0.509434     0.981481]

train: 957 test: 106
=================== Results ===================
Accuracy 0.735849
            Positive     Negative
F1         [0.611111     0.800000]
Precision  [0.846154     0.700000]
Recall     [0.478261     0.933333]

train: 957 test: 106
=================== Results ===================
Accuracy 0.801887
     